<a href="https://colab.research.google.com/github/sidharthdk/BMS-Data-pre-processing-files/blob/main/SNL_LFP_dataset_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === SNL LFP BATTERY DATASET: LOCAL ZIP → CLEANED ZIP ===
# 1) Upload your SNL LFP ZIP (e.g., SNL_LFP_raw.zip) in this cell.
# 2) Run the cell. Final file: /content/snl_lfp_cleaned.zip

from google.colab import files
import zipfile
import pandas as pd
import numpy as np
from pathlib import Path

# Step 1: Upload ZIP
print("📤 Upload your SNL LFP ZIP file (from BatteryArchive / BatteryML SNL structure):")
uploaded = files.upload()  # pick e.g. SNL_LFP.zip
zip_path = list(uploaded.keys())[0]
print(f"Uploaded: {zip_path}")

# Step 2: Extract ZIP to temp folder
raw_dir = Path("/content/snl_lfp_raw")
raw_dir.mkdir(exist_ok=True)
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(raw_dir)
print(f"✅ Extracted to {raw_dir}")

# Expected structure (per BatteryML docs): SNL_18650_LFP_*_cycle_data.csv / *_timeseries.csv[web:84]
csv_files = list(raw_dir.rglob("*.csv"))
print(f"Found {len(csv_files)} CSV files")

# Step 3: Config for SNL LFP cells (18650, 2.0–3.6 V window often used for LFP cells in this study)[web:74][web:84]
CONFIG = {
    "interp_gap": 10,      # max consecutive NaNs to interpolate
    "v_min": 2.0,          # LFP lower cutoff in SNL tests
    "v_max": 3.6,          # LFP upper cutoff
    "drop_bad_cycles": True
}

clean_dir = Path("/content/snl_lfp_cleaned")
clean_dir.mkdir(exist_ok=True)

def clean_snl_file(path: Path) -> bool:
    """Clean a single SNL LFP CSV (cycle_data or timeseries). Returns True if saved."""
    fname = path.name
    print(f"\nCleaning {fname}...")
    try:
        df_raw = pd.read_csv(path)
        if df_raw.empty:
            print("  ⚠ Empty file, skipped")
            return False
        df = df_raw.copy()
        orig_rows = len(df)

        # 1) Standardize column names
        df.columns = (
            df.columns.str.strip().str.lower()
            .str.replace(r"[^a-z0-9_]", "", regex=True)
            .str.replace(r"_+", "_", regex=True)
        )

        # 2) Ensure core SNL naming if possible (BatteryML-style)[web:84]
        rename_map = {}
        for c in df.columns:
            if c in ["cycle", "cycleindex"]: rename_map[c] = "cycle_index"
            if c in ["time_s", "testtime_s", "test_time"]: rename_map[c] = "test_time_s"
            if c in ["voltage", "v"]: rename_map[c] = "voltage_v"
            if c in ["current", "i"]: rename_map[c] = "current_a"
            if c in ["temperature", "temp"]: rename_map[c] = "cell_temperature_c"
            if c in ["capacity_ah", "q_ah", "q"]: rename_map[c] = "discharge_capacity_ah"
        df = df.rename(columns=rename_map)

        # 3) Convert numerics & basic cleaning
        df = df.apply(pd.to_numeric, errors="coerce")
        df = df.dropna(axis=1, how="all")   # drop all-NaN cols
        df = df.dropna(how="all")           # drop all-NaN rows
        df = df.drop_duplicates()

        # 4) Interpolate numeric gaps (per cycle/time)
        num_cols = df.select_dtypes(include=np.number).columns.tolist()
        if len(num_cols) > 0 and len(df) > 1:
            time_col = "test_time_s" if "test_time_s" in df.columns else None
            if time_col is not None:
                sort_cols = ["cycle_index", time_col] if "cycle_index" in df.columns else [time_col]
                df = df.sort_values(sort_cols)
            df[num_cols] = df[num_cols].interpolate(
                method="linear",
                limit=CONFIG["interp_gap"],
                limit_direction="both"
            )
            df[num_cols] = df[num_cols].ffill().bfill()
            df = df.dropna()

        # 5) Physics constraints for LFP (SNL 18650 cells)[web:74][web:77]
        if "voltage_v" in df.columns:
            df = df[(df["voltage_v"] >= CONFIG["v_min"]) & (df["voltage_v"] <= CONFIG["v_max"])]

        # Non-negative constraints for capacities/energy
        for col in num_cols:
            if any(key in col for key in ["capacity", "energy", "ah", "wh", "power"]):
                df[col] = df[col].clip(lower=0)

        # 6) Monotonic time per cycle (timeseries files)
        if CONFIG["drop_bad_cycles"] and "cycle_index" in df.columns and "test_time_s" in df.columns:
            is_bad = df.groupby("cycle_index")["test_time_s"].apply(lambda x: ~x.is_monotonic_increasing)
            bad_cycles = is_bad[is_bad].index.tolist()
            if bad_cycles:
                df = df[~df["cycle_index"].isin(bad_cycles)]
                print(f"  ⚠ Dropped {len(bad_cycles)} non-monotonic cycles")

        # 7) Save only if meaningful
        if len(df) < 10:
            print(f"  ⚠ Too few rows after cleaning ({len(df)}) → skipped")
            return False

        out_name = f"{path.stem}_cleaned.csv"
        out_path = clean_dir / out_name
        df.to_csv(out_path, index=False)
        print(f"  ✅ Saved {len(df):,} rows (retention {len(df)/orig_rows:.0%}) → {out_name}")
        return True

    except Exception as e:
        print(f"  ❌ Error: {type(e).__name__}: {str(e)[:80]}")
        return False

# Run cleaning on all CSV files
saved = 0
for p in csv_files:
    if clean_snl_file(p):
        saved += 1

print(f"\nSummary: cleaned {saved}/{len(csv_files)} files")

# Step 4: Zip cleaned files
clean_zip = "/content/snl_lfp_cleaned.zip"
with zipfile.ZipFile(clean_zip, "w", zipfile.ZIP_DEFLATED) as zf:
    for f in clean_dir.glob("*.csv"):
        zf.write(f, f.name)

print(f"\n🎉 DONE! Cleaned ZIP: {clean_zip}")
!du -sh {clean_zip} {clean_dir}


📤 Upload your SNL LFP ZIP file (from BatteryArchive / BatteryML SNL structure):


Saving SNL LFP-20260122T055953Z-1-001.zip to SNL LFP-20260122T055953Z-1-001.zip
Uploaded: SNL LFP-20260122T055953Z-1-001.zip
✅ Extracted to /content/snl_lfp_raw
Found 60 CSV files

Cleaning SNL_18650_LFP_25C_20-80_0.5-0.5C_b_timeseries.csv...
  ✅ Saved 537,880 rows (retention 100%) → SNL_18650_LFP_25C_20-80_0.5-0.5C_b_timeseries_cleaned.csv

Cleaning SNL_18650_LFP_25C_20-80_0.5-0.5C_c_cycle_data.csv...
  ✅ Saved 5,944 rows (retention 100%) → SNL_18650_LFP_25C_20-80_0.5-0.5C_c_cycle_data_cleaned.csv

Cleaning SNL_18650_LFP_15C_0-100_0.5-2C_a_cycle_data.csv...
  ✅ Saved 3,760 rows (retention 100%) → SNL_18650_LFP_15C_0-100_0.5-2C_a_cycle_data_cleaned.csv

Cleaning SNL_18650_LFP_15C_0-100_0.5-2C_b_cycle_data.csv...
  ✅ Saved 3,753 rows (retention 100%) → SNL_18650_LFP_15C_0-100_0.5-2C_b_cycle_data_cleaned.csv

Cleaning SNL_18650_LFP_25C_40-60_0.5-0.5C_a_timeseries.csv...
  ✅ Saved 552,695 rows (retention 100%) → SNL_18650_LFP_25C_40-60_0.5-0.5C_a_timeseries_cleaned.csv

Cleaning SNL_18650